In [227]:
import pandas as pd
import string
from operator import itemgetter
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from collections import Counter, OrderedDict

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

In [2]:
file_location = '/Users/hyunoochang/code/chyunoo/wineteller/raw_data/winemag-data_first150k.csv'

data = pd.read_csv(file_location, index_col='Unnamed: 0')
print(data.shape)

(150930, 10)


In [135]:
data.head(5)

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [122]:
labels = data['variety']
descriptions = data['description']

In [123]:
print('{}   :   {}'.format(labels.tolist()[0], descriptions.tolist()[0]))
print('{}   :   {}'.format(labels.tolist()[1], descriptions.tolist()[1]))
print('{}   :   {}'.format(labels.tolist()[2], descriptions.tolist()[2]))

Cabernet Sauvignon   :   This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.
Tinta de Toro   :   Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.
Sauvignon Blanc   :   Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 

In [124]:
varietal_counts = labels.value_counts()
print(varietal_counts[:50])

Chardonnay                       14482
Pinot Noir                       14291
Cabernet Sauvignon               12800
Red Blend                        10062
Bordeaux-style Red Blend          7347
Sauvignon Blanc                   6320
Syrah                             5825
Riesling                          5524
Merlot                            5070
Zinfandel                         3799
Sangiovese                        3345
Malbec                            3208
White Blend                       2824
Rosé                              2817
Tempranillo                       2556
Nebbiolo                          2241
Portuguese Red                    2216
Sparkling Blend                   2004
Shiraz                            1970
Corvina, Rondinella, Molinara     1682
Rhône-style Red Blend             1505
Pinot Gris                        1365
Barbera                           1365
Cabernet Franc                    1363
Sangiovese Grosso                 1346
Pinot Grigio             

In [125]:
print(varietal_counts[-10:])

Cococciola             1
Garnacha Tintorera     1
Malbec-Petit Verdot    1
Albarossa              1
Aidani                 1
Carignan-Syrah         1
Premsal                1
Muskat                 1
Syrah-Carignan         1
Carnelian              1
Name: variety, dtype: int64


In [161]:
test = data

In [160]:
#test = data[:10000]

In [58]:
#list(set(sentences_list) - set(sentences_tokenized))

['While remarkably lithe and delicate in style, this complex auslese is lusciously concentrated, with ripe peach and nectarine flavors balanced by hints of smoke, spice and musk',
 'The vibrant, tangy palate offers succulent nectarine, wild cherry, crushed strawberry and mineral alongside a backbone of lively acidity',
 "Full of potential, it's drinkable now but will be so much better from 2017",
 'Principally Pinot Noir, the blend includes 25% Pinot Meunier and a splash of Chardonnay',
 'It is aged in cellars in Saumur on the Loire river',
 'Firm, ripe tannins provide the framework',
 'This wine does the site justice, a smooth concoction of cherry pit and allspice, elegant and yet plenty generous in fruit and structure, measured by natural acidity',
 'The wood aging shows only lightly, leaving fruitiness at the end',
 'This 100% varietal wine offers aromas of fresh-ground coffee, chocolate, herbs and plum along with barrel accents',
 'Lush aromatics conjure up dreams of strawberry cre

In [104]:
#from tqdm.notebook import trange, tqdm

In [110]:
%%time
#reviews_list = list(test['description'])
#reviews_list = [str(r) for r in reviews_list]
#full_corpus = ' '.join(reviews_list)
#sentences_tokenized = sent_tokenize(full_corpus)
##
##print(sentences_tokenized[:5])

CPU times: user 29.3 s, sys: 13.1 s, total: 42.3 s
Wall time: 45.8 s


In [162]:
%%time
reviews_list = list(test['description'])
reviews_list = [str(r) for r in reviews_list]
sentences_tokenized=[]
for review in tqdm_notebook(reviews_list) :
    sentences_tokenized.append(sent_tokenize(review))
sentences_tokenized = [item for sublist in sentences_tokenized for item in sublist]

  0%|          | 0/150930 [00:00<?, ?it/s]

CPU times: user 18.1 s, sys: 781 ms, total: 18.8 s
Wall time: 21 s


In [163]:
len(sentences_tokenized)

417173

In [164]:
stop_words = set(stopwords.words('english')) 

punctuation_table = str.maketrans({key: None for key in string.punctuation})
sno = SnowballStemmer('english')

def normalize_text(raw_text):
    try:
        word_list = word_tokenize(raw_text)
        normalized_sentence = []
        for w in word_list:
            try:
                w = str(w)
                lower_case_word = str.lower(w)
                stemmed_word = sno.stem(lower_case_word)
                no_punctuation = stemmed_word.translate(punctuation_table)
                if len(no_punctuation) > 1 and no_punctuation not in stop_words:
                    normalized_sentence.append(no_punctuation)
            except:
                continue
        return normalized_sentence
    except:
        return ''

sentence_sample = sentences_tokenized[:10]
normalized_sentences = []
for s in sentences_tokenized:
    normalized_text = normalize_text(s)
    normalized_sentences.append(normalized_text)

In [165]:
sentences_tokenized[:10]

['This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak.',
 'Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background.',
 'Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance.',
 'Enjoy 2022–2030.',
 'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla.',
 'This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices.',
 'A toasty, everlasting finish is heady but ideally balanced.',
 'Drink through 2023.',
 'Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white.',
 'Dark gold in color, it layers toasted hazelnut, pear compote and orange peel flavors, reveling in the succulence of its 122 g/L of residual sugar.']

In [166]:
normalized_sentences[:3]

[['tremend',
  '100',
  'variet',
  'wine',
  'hail',
  'oakvill',
  'age',
  'three',
  'year',
  'oak'],
 ['juici',
  'redcherri',
  'fruit',
  'compel',
  'hint',
  'caramel',
  'greet',
  'palat',
  'frame',
  'eleg',
  'fine',
  'tannin',
  'subtl',
  'minti',
  'tone',
  'background'],
 ['balanc', 'reward', 'start', 'finish', 'year', 'ahead', 'develop', 'nuanc']]

In [167]:
phrases = Phrases(normalized_sentences)
phrases = Phrases(phrases[normalized_sentences])

ngrams = Phraser(phrases)

phrased_sentences = []
for sent in normalized_sentences:
    phrased_sentence = ngrams[sent]
    phrased_sentences.append(phrased_sentence)

full_list_words = [item for sublist in phrased_sentences for item in sublist]

In [168]:
phrased_sentences[:5]

[['tremend',
  '100_variet',
  'wine',
  'hail',
  'oakvill',
  'age',
  'three_year',
  'oak'],
 ['juici',
  'redcherri',
  'fruit',
  'compel',
  'hint',
  'caramel',
  'greet',
  'palat',
  'frame',
  'eleg',
  'fine',
  'tannin',
  'subtl',
  'minti',
  'tone',
  'background'],
 ['balanc', 'reward', 'start', 'finish', 'year', 'ahead', 'develop', 'nuanc'],
 ['enjoy', '2022–2030'],
 ['ripe',
  'aroma',
  'fig',
  'blackberri',
  'cassi',
  'soften_sweeten',
  'slather',
  'oaki',
  'chocol',
  'vanilla']]

In [169]:
full_list_words[:5]

['tremend', '100_variet', 'wine', 'hail', 'oakvill']

In [170]:
word_counts = Counter(full_list_words)
sorted_counts = OrderedDict(word_counts.most_common(5000))
counter_df = pd.DataFrame.from_dict(sorted_counts, orient='index')
top_5000_words = counter_df.head(5000)
counter_df.to_csv('top_5000_descriptors.csv')

In [171]:
top_5000_words

,0
wine,95170
flavor,86401
fruit,68895
finish,45375
acid,40144
...,...
char_beef,35
toast_brioch,35
superconcentr,35
thrust,35


In [172]:
map_location = '/Users/hyunoochang/code/chyunoo/wineteller/raw_data/descriptor_mapping.csv'

In [173]:
descriptor_mapping = pd.read_csv(map_location).set_index('raw descriptor')
descriptor_mapping

,level_3,level_2,level_1
raw descriptor,,,
abras,abrasive,high_tannin,tannin
acacia,acacia,flowery,flower
acacia_flower,acacia,flowery,flower
aciddriven,acid_driven,high_acid,acid
aggress,aggressive,high_acid,acid
...,...,...,...
zest,zest,citrus_fruit,fruit
zesti,zesty,high_acid,acid
zing,zingy,high_acid,acid


In [174]:
descriptor_list = ['body', 'finish', 'complexity']

filtered_descriptor_mapping = descriptor_mapping[descriptor_mapping['level_1'].isin(descriptor_list)]

In [175]:
filtered_descriptor_mapping

,level_3,level_2,level_1
raw descriptor,,,
airi,airy,light_bodied,body
big_bold,bold,full_bodied,body
bullish,bullish,full_bodied,body
blocki,chunky,full_bodied,body
chunki,chunky,full_bodied,body
...,...,...,...
unoak,unoaked,light_bodied,body
viscos,viscous,full_bodied,body
viscous,viscous,full_bodied,body


In [176]:
filtered_descriptor_mapping["level_1"].value_counts()

body          57
complexity    16
finish         6
Name: level_1, dtype: int64

In [177]:
list(filtered_descriptor_mapping.index)

['airi',
 'big_bold',
 'bullish',
 'blocki',
 'chunki',
 'solid_chunki',
 'clampi',
 'close',
 'complex',
 'complic',
 'dainti',
 'deep',
 'depth',
 'eas',
 'easi_sip',
 'eleg',
 'expans',
 'extract',
 'feminin',
 'finess',
 'ampl_weight',
 'bold',
 'full_bodi',
 'fullbodi',
 'soupi',
 'weighti',
 'hearti',
 'heavi',
 'heavier',
 'heavyweight',
 'lavish',
 'rather_lean',
 'length',
 'lengthi',
 'long_mouthwat',
 'longlast',
 'lightest',
 'light',
 'light_bodi',
 'lightbodi',
 'lighter',
 'lighter_style',
 'lightfoot',
 'lightweight',
 'thin',
 'linear',
 'straightforward',
 'lush',
 'luxuri',
 'mass',
 'medium_bodi',
 'medium_fullbodi',
 'medium_weight',
 'mediumbodi',
 'mediumweight',
 'medium_length',
 'mediumlength_finish',
 'modest',
 'onedimension',
 'opul',
 'plump',
 'quaffer',
 'refin',
 'rich',
 'richer',
 'robust',
 'uncompl',
 'stout',
 'sturdi',
 'succul',
 'superrich',
 'syrup',
 'syrupi',
 'thick',
 'unoak',
 'viscos',
 'viscous',
 'voluptu',
 'weight']

In [178]:
#filtered_descriptor_mapping["level_3"]["light"]

In [179]:
def return_mapped_descriptor(word):
    if word in list(filtered_descriptor_mapping.index):
        normalized_word = filtered_descriptor_mapping['level_3'][word]
        return normalized_word
    else:
        return ""

normalized_sentences = []
for sent in phrased_sentences:
    normalized_sentence = []
    for word in sent:
        normalized_word = return_mapped_descriptor(word)
        if normalized_word != "" :
            normalized_sentence.append(str(normalized_word))
        else :
            pass
    normalized_sentences.append(normalized_sentence)

In [189]:
phrased_sentences[:10]

[['tremend',
  '100_variet',
  'wine',
  'hail',
  'oakvill',
  'age',
  'three_year',
  'oak'],
 ['juici',
  'redcherri',
  'fruit',
  'compel',
  'hint',
  'caramel',
  'greet',
  'palat',
  'frame',
  'eleg',
  'fine',
  'tannin',
  'subtl',
  'minti',
  'tone',
  'background'],
 ['balanc', 'reward', 'start', 'finish', 'year', 'ahead', 'develop', 'nuanc'],
 ['enjoy', '2022–2030'],
 ['ripe',
  'aroma',
  'fig',
  'blackberri',
  'cassi',
  'soften_sweeten',
  'slather',
  'oaki',
  'chocol',
  'vanilla'],
 ['full',
  'layer',
  'intens',
  'cushion',
  'palat',
  'rich',
  'flavor',
  'chocolati',
  'black',
  'fruit',
  'bake',
  'spice'],
 ['toasti', 'everlast', 'finish', 'headi', 'ideal', 'balanc'],
 ['drink_2023'],
 ['mac',
  'watson',
  'honor',
  'memori',
  'wine',
  'onc',
  'made',
  'mother',
  'tremend',
  'delici',
  'balanc',
  'complex',
  'botrytis',
  'white'],
 ['dark',
  'gold_color',
  'layer',
  'toast_hazelnut',
  'pear',
  'compot',
  'orang_peel',
  'flavor',
 

In [180]:
normalized_sentences[:10]

[[], ['elegant'], [], [], [], ['rich'], [], [], ['complex'], ['succulent']]

In [219]:
model = Word2Vec(normalized_sentences, vector_size=300, min_count=1, epochs=15)
print(model)

model.save('model.bin')

Word2Vec<vocab=50, vector_size=300, alpha=0.025>


In [220]:
model.wv.index_to_key

['rich',
 'light_bodied',
 'complex',
 'elegant',
 'full_bodied',
 'depth',
 'medium_bodied',
 'lush',
 'closed',
 'heavy',
 'thick',
 'weight',
 'opulent',
 'extracted',
 'chunky',
 'plump',
 'length',
 'modest',
 'refined',
 'syrupy',
 'low_complexity',
 'succulent',
 'robust',
 'linear',
 'finessed',
 'lengthy',
 'hearty',
 'sturdy',
 'one_dimensional',
 'voluptuous',
 'bold',
 'feminine',
 'luxurious',
 'lavish',
 'viscous',
 'complicated',
 'medium_length_finish',
 'unoaked',
 'simple',
 'mass',
 'super_rich',
 'quaffer',
 'expansive',
 'easy',
 'stout',
 'light',
 'clampy',
 'dainty',
 'airy',
 'bullish']

In [224]:
model.wv.most_similar(positive='light_bodied', topn=50)

[('medium_bodied', 0.9927793741226196),
 ('linear', 0.9914177060127258),
 ('modest', 0.9914150834083557),
 ('feminine', 0.9908044934272766),
 ('low_complexity', 0.9882341027259827),
 ('length', 0.9881134629249573),
 ('refined', 0.9871636629104614),
 ('medium_length_finish', 0.9871259927749634),
 ('finessed', 0.9865640997886658),
 ('airy', 0.9841231107711792),
 ('quaffer', 0.9826445579528809),
 ('simple', 0.9819291830062866),
 ('easy', 0.981232762336731),
 ('dainty', 0.9811624884605408),
 ('elegant', 0.9809308648109436),
 ('unoaked', 0.9801408052444458),
 ('plump', 0.978657603263855),
 ('light', 0.9771679043769836),
 ('one_dimensional', 0.9764285087585449),
 ('complex', 0.9758601188659668),
 ('clampy', 0.9754361510276794),
 ('closed', 0.9751891493797302),
 ('depth', 0.9746557474136353),
 ('lengthy', 0.9735859632492065),
 ('sturdy', 0.9732953906059265),
 ('super_rich', 0.9732057452201843),
 ('voluptuous', 0.9725902080535889),
 ('complicated', 0.9716777801513672),
 ('weight', 0.9716350436

In [211]:
#takes ~10min
wine_reviews = list(data['description'])

def return_descriptor_from_mapping(word):
    if word in list(filtered_descriptor_mapping.index):
        descriptor_to_return = filtered_descriptor_mapping['level_3'][word]
        return descriptor_to_return

descriptorized_reviews = []
for review in wine_reviews:
    normalized_review = normalize_text(review)
    phrased_review = ngrams[normalized_review]
    descriptors_only = [return_descriptor_from_mapping(word) for word in phrased_review]
    no_nones = [str(d) for d in descriptors_only if d is not None]
    descriptorized_review = ' '.join(no_nones)
    descriptorized_reviews.append(descriptorized_review)

In [213]:
descriptorized_reviews[:10]

['elegant',
 'rich',
 'complex succulent',
 '',
 'rich',
 'depth',
 '',
 'lush lush depth',
 'chunky',
 'succulent lush length']

In [247]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit(descriptorized_reviews)

dict_of_tfidf_weightings = dict(zip(X.get_feature_names(), X.idf_))

wine_review_vectors = []
for d in tqdm_notebook(descriptorized_reviews):
    descriptor_count = 0
    weighted_review_terms = []
    terms = d.split(' ')
    for term in terms:
        if term in dict_of_tfidf_weightings.keys():
            tfidf_weighting = dict_of_tfidf_weightings[term]
            word_vector = model.wv.get_vector(term).reshape(1, 300)
            weighted_word_vector = tfidf_weighting * word_vector
            weighted_review_terms.append(weighted_word_vector)
            descriptor_count += 1
        else:
            continue
    try:
        review_vector = sum(weighted_review_terms)/len(weighted_review_terms)
    except:
        review_vector = []
    vector_and_count = [terms, review_vector, descriptor_count]
    wine_review_vectors.append(vector_and_count)

data['normalized_descriptors'] = list(map(itemgetter(0), wine_review_vectors))
data['review_vector'] = list(map(itemgetter(1), wine_review_vectors))
data['descriptor_count'] = list(map(itemgetter(2), wine_review_vectors))

data.reset_index(inplace=True)


/Users/hyunoochang/.pyenv/versions/3.8.12/envs/wineteller/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


  0%|          | 0/150930 [00:00<?, ?it/s]

In [245]:
data.head()

,index,country,description,designation,points,price,province,region_1,region_2,variety,winery,normalized_descriptors,review_vector,descriptor_count
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,[elegant],"[[-0.07686549, 0.118861645, -0.07442225, 0.172...",1
1,0,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,[rich],"[[-0.058690846, 0.14173521, 0.0077731507, 0.11...",1
2,0,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"[complex, succulent]","[[-0.06216684, 0.30808628, -0.08620003, 0.0290...",2
3,0,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,[],[],0
4,0,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,[rich],"[[-0.058690846, 0.14173521, 0.0077731507, 0.11...",1


In [254]:
data.loc[1,"description"]

'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.'

In [256]:
data[data["descriptor_count"]>0]

,level_0,index,country,description,designation,points,price,province,region_1,region_2,variety,winery,normalized_descriptors,review_vector,descriptor_count
0,0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,[elegant],"[[-0.07686549, 0.118861645, -0.07442225, 0.172...",1
1,1,0,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez,[rich],"[[-0.058690846, 0.14173521, 0.0077731507, 0.11...",1
2,2,0,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,"[complex, succulent]","[[-0.06216684, 0.30808628, -0.08620003, 0.0290...",2
4,4,0,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude,[rich],"[[-0.058690846, 0.14173521, 0.0077731507, 0.11...",1
5,5,0,Spain,"Deep, dense and pure from the opening bell, th...",Numanthia,95,73.0,Northern Spain,Toro,NaN,Tinta de Toro,Numanthia,[depth],"[[-0.101349674, 0.25462893, -0.04930349, 0.099...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150922,150922,0,Italy,Made by 30-ish Roberta Borghese high above Man...,Superiore,91,NaN,Northeastern Italy,Colli Orientali del Friuli,NaN,Tocai,Ronchi di Manzano,[medium_bodied],"[[-0.014577129, 0.18955237, -0.07719464, 0.082...",1
150923,150923,0,France,"Rich and toasty, with tiny bubbles. The bouque...",Demi-Sec,91,30.0,Champagne,Champagne,NaN,Champagne Blend,Jacquart,[rich],"[[-0.058690846, 0.14173521, 0.0077731507, 0.11...",1
150924,150924,0,France,"Really fine for a low-acid vintage, there's an...",Diamant Bleu,91,70.0,Champagne,Champagne,NaN,Champagne Blend,Heidsieck & Co Monopole,"[closed, complex]","[[-0.052041233, 0.23237354, -0.059730045, 0.03...",2
150925,150925,0,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio,[elegant],"[[-0.07686549, 0.118861645, -0.07442225, 0.172...",1
